# Synthetic Image Data Generator

Here is a very basic Demo of Synthetic Image Data generation. Run each code block in sequence to generate images. This will generate 3 images based upon the prompt below. 

You will need to select the T4 GPU in the change runtime menu. Click the arrown next to RAM and DISK. Select CHANGE RUNTIME TYPE and select the T4 GPU. Then run the code blocks.

In [ ]:
!pip install --quiet --upgrade diffusers transformers accelerate mediapy peft

Import the model and other requirements

In [ ]:
import mediapy as media
import random
import sys
import torch

from diffusers import DiffusionPipeline, TCDScheduler
from huggingface_hub import hf_hub_download

# Choose either 8 or 12 steps:
num_inference_steps = 12

base_model_id = "stabilityai/stable-diffusion-xl-base-1.0"
repo_name = "ByteDance/Hyper-SD"
plural = "s" if num_inference_steps > 1 else ""
ckpt_name = f"Hyper-SDXL-{num_inference_steps}step{plural}-CFG-lora.safetensors"
device = "cuda"

pipe = DiffusionPipeline.from_pretrained(base_model_id, torch_dtype=torch.float16, variant="fp16").to(device)
pipe.load_lora_weights(hf_hub_download(repo_name, ckpt_name))
pipe.fuse_lora()
pipe.scheduler = TCDScheduler.from_config(pipe.scheduler.config)

Here you can change the prompt for the image you want to generate.  

In [ ]:
prompt = " a photo of a Road Sweeper Used to clean the roads and construction areas."

Run this code block to see you images generated.

In [ ]:
import random
import sys
import torch

num_images = 3 # Set the number of images to generate
num_inference_steps = 40  # Ensure this variable is defined
guidance_scale = 9.0
eta = 0.5
path= 'images/'
generator = torch.Generator(device)
seed = random.randint(0, sys.maxsize)
generator.manual_seed(seed)

images = []
for i in range(num_images):
    generator.manual_seed(seed + i)  # Change seed for variation
    image = pipe(
        prompt=prompt,
        num_inference_steps=num_inference_steps,
        guidance_scale=guidance_scale,
        eta=eta,
        generator=generator,
    ).images[0]
    images.append(image)
    image.save(f"output_{i}.png")


print(f"Prompt:\t{prompt}\nSeed:\t{seed}")
media.show_images(images)


